In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB



In [ ]:
main_df = pd.read_csv('data/train.csv')

In [ ]:
# First 5 data samples for reference
main_df.head(5)

In [ ]:
# Exploratory Data Analysis (EDA)

In [ ]:
main_df.shape
# Main dataframe has 891 training data with 12 features

In [ ]:
main_df.info()
# Most features are numerical except for ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

In [ ]:
main_df.isna().sum()

# There a 2 missing values in the feature 'Embarked', 
# about 20% of the Ages are missing, and 
# the majority of the Cabins are missing

In [ ]:
# Dealing with missing values (Embarked)

main_df['Embarked'].value_counts()
# Since there are only 2 missing values for Embarked, filling the missing value with the mode 
# of the dataset is reasonable. In this case we fill it with 'S' which takes account about 70% of 'Embarked'\
main_df['Embarked'].fillna('S', inplace=True)


In [ ]:
# Dealing with missing values (Age)
# To fill the missing age value as accurately as possible, I would use the passanger's title/salutation as a 
# benchmark. I first create a new column by applying a lambda function that extracts the passanger's title, 
# taking advantage of the consistent string structure
main_df['Salutation'] = main_df.Name.apply(lambda name: name.split(',')[1].split('.')[0].strip())
main_df_subage = main_df.groupby(['Salutation'])
# We created a sub data frame that groups the main dataframe by the passanger's salutation, we then describe 
# the dataframe, giving us insights to how to fill in the corresponding missing values
main_df_subage.describe()
main_df_subage.hist('Age', by='Salutation')

# Since not all ages are normally distributed according to their titles, we would fill the missing value with
# the median age of each title

# Filling missing values

# Calculate median age for each salutation
median_age_by_salutation = main_df_subage['Age'].median()

# Define a function to fill missing age values based on salutation
def fill_missing_age(row):
    if pd.isnull(row['Age']):
        return median_age_by_salutation[row['Salutation']]
    else:
        return row['Age']

# Apply the function to fill missing age values
main_df['Age'] = main_df.apply(fill_missing_age, axis=1)

In [ ]:
# Dealing with missing values (Cabin)
main_df['Cabin'].unique()
main_df['Cabin'].value_counts()

# After analyzing the value counts and unique cabin values, it is very difficult to come up with a way to methodically
# fill the missing values without sacrificing accuracy. Since cabin numbers are unique (except for family who might 
# be in the same cabin), it is not reasonable to fill the missing value with the mode or most common cabin. 
# With this in mind, we will make the missing cabins as 'Unknown'

main_df['Cabin'].fillna('Unknown', inplace=True)

In [ ]:
# Now that missing value has been handled, let's move on to feature engineering

# --------------------------------------------------------------------------------

# First thing is to drop the 'Name' column because it is highly irrelevant to survival 
# +main_df.drop(columns=['Name'], inplace=True)

# --------------------------------------------------------------------------------




In [ ]:
# --------------------------------------------------------------------------------

# We would keep 'Survived' for it is the target variable 

# --------------------------------------------------------------------------------

In [ ]:
# --------------------------------------------------------------------------------

# Next, let us analyze the feature Pclass 

# Create a pivot table that calculates survival mean of each passanger depending on their 'Pclass'
pivot_table = main_df.pivot_table(index='Pclass', values='Survived')
sns.heatmap(pivot_table, cmap='viridis', annot=True, fmt=".2f", linewidths=0.5)

# Finds the correlation matrix between Pclass and Survived
correlation_matrix = main_df[['Pclass'] + ['Survived']].corr()

# Plot heatmap using seaborn
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, cmap='coolwarm', annot=True, fmt=".2f", linewidths=0.5)

# INSIGHT: There seemed to be a downward trend of 'Survived' as 'Pclass' goes up. Additionally,
# correlation matrix shows a moderate negative linear correlation betweeen 'Pclass' and our target 
# variable, 'Survived'. This means that this feature is relevant and will be kept for our model

# --------------------------------------------------------------------------------

In [ ]:
# --------------------------------------------------------------------------------

# Next, let us analyze the feature 'Sex'
# First we need to apply Label Encoder to convert this categorical column into a numerical one

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Encode 'Sex' column

main_df['Sex_Encoded'] = label_encoder.fit_transform(main_df['Sex'])
# Encoded male = 1, Encoded female = 0

# Create a pivot table that calculates survival mean of each passanger depending on their 'Sex'
pivot_table = main_df.pivot_table(index='Sex_Encoded', values='Survived')
sns.heatmap(pivot_table, cmap='viridis', annot=True, fmt=".2f", linewidths=0.5)

# Finds the correlation matrix between Sex and Survived
correlation_matrix = main_df[['Sex_Encoded'] + ['Survived']].corr()

# Plot heatmap using seaborn
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, cmap='coolwarm', annot=True, fmt=".2f", linewidths=0.5)

# INSIGHT: In this feature, we found a strong negative correlation between sex and survival. Moreover,
# we also see a heavy dip in survival from female (0) to male (1). Therefore, 'Sex' is indeed an important
# feature for our model

# --------------------------------------------------------------------------------


In [ ]:
# --------------------------------------------------------------------------------

# Next, let us analyze age
# We need to group ages into bins since it is not practical to construct a pivot table for EVERY single
# age in the dataframe. Let us divide our age group into 8 bins and plot the corresponding heatmap

age_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80]
main_df['AgeGroup'] = pd.cut(main_df['Age'], bins=age_bins)
pivot_table = main_df.pivot_table(index='AgeGroup', values='Survived')
sns.heatmap(pivot_table, cmap='viridis', annot=True, fmt=".2f", linewidths=0.5)

# Next, we can plot the correlation heatmap between 'Age' and output variable 

# Finds the correlation matrix between Age and Survived
correlation_matrix = main_df[['Age'] + ['Survived']].corr()

# Plot heatmap using seaborn
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, cmap='coolwarm', annot=True, fmt=".2f", linewidths=0.5)

# INSIGHT: From these two heatmaps, we actually got two conflicting result. The first heatmap from the pivot table shows us that 
# younger people tend to survive, hence the left skew. The correlation matrix however, states that there is almost no linear corelation at all between
# the two variables. To further confirm this, I will investigate by creating a sub-feature called AgeGroup, to further highlight the decrease trend
# in survival as age increases 

# Define age groups
age_bins = [0, 4, 13, 19, 24, 65, main_df['Age'].max()]
age_labels = ['Infant', 'Child', 'Teen', 'Young Adult', 'Adult', 'Elderly']
age_group_mapping = {'Infant': 0, 'Child': 1, 'Teen': 2, 'Young Adult': 3, 'Adult': 4, 'Elderly': 5}

# Create 'AgeGroup' column based on age groups (binning)
main_df['AgeGroup'] = pd.cut(main_df['Age'], bins=age_bins, labels=age_labels)

# Calculate survival rate by age group
survival_rate_by_age_group = main_df.groupby('AgeGroup')['Survived'].mean()
print(survival_rate_by_age_group)

# INSIGHT: We indeed do see a negative correlation between age groups with younger passangers having a higher survival chance. It also seemed like the 
# decrease is not linear which caused the correlation matrix to be a weak linear negative correlation. However, by analyzing the trend, age indeed is a
# relevant feature for predicting survival rate

# --------------------------------------------------------------------------------


In [ ]:
# --------------------------------------------------------------------------------

# Next we will analyze the feature SibSp (# sib/spouses) and Parch (# of parents)

# Pivot table that reveals the mean of Survived in each category of Parch and SibSp
pivot_table = main_df.pivot_table(index='Parch', values='Survived')
sns.heatmap(pivot_table, cmap='viridis', annot=True, fmt=".2f", linewidths=0.5)

pivot_table = main_df.pivot_table(index='SibSp', values='Survived')
sns.heatmap(pivot_table, cmap='viridis', annot=True, fmt=".2f", linewidths=0.5)

# INSIGHT: After observing the mean survival for both Parch and SibSp, there seemed to be no trend in
# how the two features moves with 'Survived'. To further confirm this, let us plot the correlation matrix
# between the two features against 'Survived'


# Finds the correlation matrix between Parch and Survived
correlation_matrix = main_df[['Parch'] + ['Survived']].corr()

# Plot heatmap using seaborn
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, cmap='coolwarm', annot=True, fmt=".2f", linewidths=0.5)

# Finds the correlation matrix between SibSp and Survived
correlation_matrix = main_df[['SibSp'] + ['Survived']].corr()

# Plot heatmap using seaborn
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, cmap='coolwarm', annot=True, fmt=".2f", linewidths=0.5)

# As hypothesized, there is no linear correlation between having more sibling/parents/spouses/children on board
# with survival, meaning that the variation in survival rate between groups happened by chance. However, I do want
# to further explore why the higher SibSp and Parch group has 0 survival chance

main_df[(main_df['Parch'] > 3) | (main_df['SibSp'] > 4)]

# This confirms our hypothesis that such data group are considered as outliers since there are only select numbers
# of passanger who has high SibSp or Parch. Another thing to note is that all of them has Pclass value of 3, meaning
# that Pclass more than likely played a bigger role than SibSp and Parch

# INSIGHT: If we discard the outliers (high value SibSp and Parch), we would get a pretty much 50% survival rate
# from both SibSp and Parch across all groups meaning that it is probably safe to discard these features from our model

# --------------------------------------------------------------------------------

In [ ]:
# --------------------------------------------------------------------------------

# Next, let us take a deeper dive into the Ticket and Fare feature

# I want to explore whether or not higher fare increases the survival rate of passangers
main_df['Fare'].describe()
main_df.loc[(main_df['Fare'] > main_df['Fare'].mean()), 'Survived'].value_counts(normalize=True)

# INSIGHT: Sure enough, the higher the fare the higher the survival chance. Though not exactly linear, the two 
# variables definitely move together.

# Next is the trickier part, we want to see how having different Tickets might affect your survival in the Titanic
# My first thought is that more expensive ticket might be marked with longer length or having a prefix. This issue might 
# cause multicolinearity. To confirm this, I did some data manipulation and created a pivot table that illustrate
# the average fare for each ticket length

sub_df = main_df
sub_df['TicketLen'] = main_df['Ticket'].apply(len)
# Finds the correlation matrix between SibSp and Survived
pivot_table = sub_df.pivot_table(index='TicketLen', values='Fare')
sns.heatmap(pivot_table, cmap='viridis', annot=True, fmt=".2f", linewidths=0.5)

# There is a spike in price mean at Tickenlen == 8, I would like to investigae it further whether it is
# the case that ticket with length 8 is more expensive or this is simply an outlier

sub_df.loc[(sub_df['TicketLen'] == 8), ['Fare']].hist()

# Sure enough, this case seemed to be an outlier as visualized by the histogram above
# To further confirm this, let's see if there is a correlation between TicketLen and Fare using heatmap correlation matrix

# Finds the correlation matrix between TicketLen and Fare
correlation_matrix = sub_df[['TicketLen'] + ['Fare']].corr()

# Plot heatmap using seaborn
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, cmap='coolwarm', annot=True, fmt=".2f", linewidths=0.5)

# INSIGHT: There is no correlation between TicketLen and Fare, which means that we still need to further engineer
# the feature Ticket and extract what attribute of the ticket might be correlated to survival (maybe the presence of letter,
# certain number prefix, combination, etc). But for this model, we can simply drop Ticket feature while keeping Fare

# --------------------------------------------------------------------------------

In [ ]:
# --------------------------------------------------------------------------------
# Now we wil explore the Cabin feature
main_df['Cabin'].unique()

# There are too many unique cabin values to work with, let us create a new feature that only takes the first 
# letter (or the class) of the cabin called 'CabinLetter'.

main_df['CabinLetter'] = main_df['Cabin'].str[0]

# Constructs a pivot table to see how the survival rate is distributed across different cabin sections
pivot_table = sub_df.pivot_table(index='CabinLetter', values='Survived')
sns.heatmap(pivot_table, cmap='viridis', annot=True, fmt=".2f", linewidths=0.5)

# INSIGHT: Passangers from cabin B, D, E, and F have relatively high survival chance compared to the others
main_df['Cabin_Encoded'] = label_encoder.fit_transform(main_df['CabinLetter'])

# Let us encode the CabinLetter so we can convert the categorical value to 
# a numerical one to visualize the correlation between CabinLetter to Survived 

# Calculate the correlation matrix between Cabin_Encoded and Survived
correlation_matrix = sub_df[['Cabin_Encoded'] + ['Survived']].corr()

# Plot heatmap using seaborn
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, cmap='coolwarm', annot=True, fmt=".2f", linewidths=0.5)

# There is a moderate negative linear correlation between CabinLetter and Survived, meaning that we will take Cabin_Encoded 
# as our selected feature
# --------------------------------------------------------------------------------

In [ ]:
# --------------------------------------------------------------------------------
# Finally, let us analyze the feature Embarked

# Let's apply the same technique as we did with Cabin, simply construct a pivot table to see the overall 
# distribution first

# Constructs a pivot table to see how the survival rate is distributed across different cabin sections
pivot_table = sub_df.pivot_table(index='Embarked', values='Survived')
sns.heatmap(pivot_table, cmap='viridis', annot=True, fmt=".2f", linewidths=0.5)

# INSIGHT: It seemed like passangers who are embarked from C, O, and S has a descending survival rate respectively
# Though the trend appears to be nonlinear, let's encode these values and consutrct a correlation matrix to further confirm this.

main_df['Embarked_Encoded'] = label_encoder.fit_transform(main_df['Embarked'])

# Calculate the correlation matrix between Embraked_Encoded and Survived
correlation_matrix = sub_df[['Embarked_Encoded'] + ['Survived']].corr()

# Plot heatmap using seaborn
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, cmap='coolwarm', annot=True, fmt=".2f", linewidths=0.5)

# There seemed to be a somewhat negative linear corrleation between survival and from where the passanger
# embarked. For this version of the model, let's include this as our feature

In [ ]:
# Begin model training

training_x = main_df[['Pclass', 'Sex_Encoded', 'Age', 'Fare', 'Cabin_Encoded', 'Embarked_Encoded']]
training_y = main_df['Survived']
X_train, X_test, y_train, y_test = train_test_split(training_x, training_y, test_size=0.2, random_state=42)

# Define models to evaluate
models = [
    LogisticRegression(),
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    GradientBoostingClassifier(),
    SVC(),
    GaussianNB(),
    XGBClassifier()
]

# Evaluate models using cross-validation on training data
best_model = None
best_score = float('-inf')

for model in models:
    scores = cross_val_score(model, X_train, y_train, cv=5)
    mean_score = np.mean(scores)

    if mean_score > best_score:
        best_model = model
        best_score = mean_score

# Train the best model on the full training data
best_model.fit(X_train, y_train)

# Evaluate the best model on the test set
test_score = accuracy_score(y_test, best_model.predict(X_test))

print(f"Best Model: {type(best_model).__name__}")
print(f"Cross-validation Mean Accuracy: {best_score:.4f}")
print(f"Test Accuracy: {test_score:.4f}")

# INSIGHT: Gradient Boosting Classifier model gives the highest predictive accuracy of ~0.81


In [ ]:
# Now, we will apply the model on the testing data and convert our result to csv to submit

test_df = pd.read_csv('data/test.csv')

test_df

In [ ]:
# Apply the same data processing methods
# Fill missing age

test_df.isna().sum()

test_df['Salutation'] = test_df.Name.apply(lambda name: name.split(',')[1].split('.')[0].strip())
test_subage = test_df.groupby(['Salutation'])


median_age_by_salutation = test_subage['Age'].median()
def fill_missing_age(row):
    if pd.isnull(row['Age']):
        return median_age_by_salutation[row['Salutation']]
    else:
        return row['Age']
test_df['Age'] = test_df.apply(fill_missing_age, axis=1)

# The only Ms in the dataset has a missing value, so we will use the median from the training set

test_df.loc[88, ['Age']] = [21]


# Fill missing Cabin

test_df['Cabin'].fillna('Unknown', inplace=True)

# Fill missing fare with median 

test_df['Fare'].fillna(test_df['Fare'].median(), inplace=True)

# Encode Sex, CabinLetter, and Embarked

label_encoder = LabelEncoder()
test_df['Sex_Encoded'] = label_encoder.fit_transform(test_df['Sex'])\

test_df['CabinLetter'] = test_df['Cabin'].str[0]
test_df['Cabin_Encoded'] = label_encoder.fit_transform(test_df['CabinLetter'])

test_df['Embarked_Encoded'] = label_encoder.fit_transform(test_df['Embarked'])

In [ ]:
x_pred_test = test_df[['Pclass', 'Sex_Encoded', 'Age', 'Fare', 'Cabin_Encoded', 'Embarked_Encoded']]
y_res = best_model.predict(x_pred_test)

submission = pd.DataFrame({
    "PassengerId" : test_df['PassengerId'],
    "Survived" : y_res
})

submission.to_csv("titanicsubmissionrf.csv", index=False)

In [ ]:
# Model subset features = ['Pclass', 'Sex_Encoded', 'Age', 'Fare', 'Cabin_Encoded', 'Embarked_Encoded']
# Best model: Gradient Boosting Classifier
# Data processing and Cleaning:
# - Missing ages replaced with the median of the passenger's corresponding salutation
# - Missing Cabin replaced to Unknown
# - Missing embarked replaced with the mode of the dataset
# - Missing Fare replaced with the median of the dataset
# - Sex encoded with label encoder
# - CabinLetter column created by taking the first string of the cabin then encoded with label encoder
# - Embarked encoded with label encoder

# The final submission results in ~0.8 accuracy